In [1]:
import pafy
import vlc
import time
from ctypes import *
import pprint
#cdll.LoadLibrary("libvlc.so.5") 
#libvlc = CDLL("libvlc.so.5")

url = "https://www.youtube.com/watch?v=lKrJp9_UB4M"
video = pafy.new(url)
#best = video.getbest()
best = video.getbestaudio() # you can get audio only!
#Pafy.getbestaudio([preftype="any"][, ftypestrict=True])
playurl = best.url
#playurl = 'https://r4---sn-3u-bh2s.googlevideo.com/videoplayback?keepalive=yes&ipbits=0&txp=5411222&clen=3461799&key=yt6&requiressl=yes&sparams=clen%2Cdur%2Cei%2Cgir%2Cid%2Cinitcwndbps%2Cip%2Cipbits%2Citag%2Ckeepalive%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Crequiressl%2Csource%2Cexpire&gir=yes&expire=1540887699&lmt=1540196015878161&c=WEB&ei=MsDXW7H6NIm6qQGv-ae4Aw&mt=1540865977&fvip=3&dur=213.861&id=o-AGzhDFjJD_1F8EM5Et34TBLDD0sTvwh7gcGmx0j4v6x4&initcwndbps=1213750&pl=19&source=youtube&mv=m&mime=audio%2Fwebm&ms=au%2Conr&ip=218.153.172.25&mm=31%2C26&mn=sn-3u-bh2s%2Csn-i3b7knld&itag=251&signature=D24438B1633A1445C373DD7D11BAC20DFBCB39CA.5BA3ECD482F83BBE33C26EAACCC87900AF6CBF72&ratebypass=yes?start=12?end=24'

In [3]:

# libvlc.libvlc_media_player_get_time(player) --> ms
Instance = vlc.Instance()

player = Instance.media_player_new()
Media = Instance.media_new(playurl)
Media.get_mrl()
player.set_media(Media)

player.play()
vlc.libvlc_media_player_set_position(player,c_float(0.0)) # set start point
eq = vlc.libvlc_audio_equalizer_new_from_preset(0)
vlc.libvlc_media_player_set_equalizer(player,eq)
time.sleep(1)

while True:
    if vlc.libvlc_media_player_get_time(player)>30000:
        print(vlc.libvlc_media_player_get_position(player))
        time.sleep(1)
        print(vlc.libvlc_media_player_get_time(player))
        print('X')
        player.stop()
        break;


0.12380792945623398
31241
X


In [ ]:
#player.pause() #-- to pause video
#player.resume()  #-- resume paused video.
player.stop()  #-- to stop/end video

In [51]:

print('196101')
print(video.duration.split(":")) #HHMMSS
# this is approximation
# if you can make this in ms 
mss = (int(video.duration.split(":")[0])*60*60+int(video.duration.split(":")[1])*60+int(video.duration.split(":")[2]))*1000
print(mss)

#print(playurl)
#Media.get_duration()

196101
['00', '06', '13']
373000


In [4]:
#songblock = {"url":"https://www.youtube.com/watch?v=xoSRIi-SnDc","start":"00:00:13","end":"00:02:01"}
def tomillisecond(timestamp):
    T = timestamp.split(':')
    return (int(T[0])*60*60+int(T[1])*60+int(T[2]))*1000

class songBlock:
    def __init__(self,songblock):
        self.url = songblock["url"]
        self.id = self.url.split("?v=")[1]
        self.video = pafy.new(songblock["url"])
        try:
            self.local_start = songblock["start"]
        except:
            self.local_start = "00:00:00"
        try:
            self.local_end = songblock["end"]
        except:
            self.local_end = self.video.duration
        
        self.global_start = "00:00:00"
        self.global_end = "00:00:00"
    
    def play(self):
        s_pos = tomillisecond(self.local_start)/tomillisecond(self.video.duration)
        e_pos = tomillisecond(self.local_end)/tomillisecond(self.video.duration)
        s_dur = tomillisecond(self.local_end)-tomillisecond(self.local_start)    
        
        best = self.video.getbestaudio() # you can get audio only!
        playurl = best.url
        Instance = vlc.Instance()
        player = Instance.media_player_new()
        Media = Instance.media_new(playurl)
        Media.get_mrl()
        player.set_media(Media)
        player.play()
        vlc.libvlc_media_player_set_position(player,c_float(s_pos)) # set start point
        time.sleep(0.1)
        while True:
            #if vlc.libvlc_media_player_get_time(player)>tomillisecond(self.local_end):
            if vlc.libvlc_media_player_get_position(player)>=e_pos:
                print("songblock {} finished".format(self.id))
                player.stop()
                break;
                
    def loop(self,count): # seems more smooth than { self.play x count }
        s_pos = tomillisecond(self.local_start)/tomillisecond(self.video.duration)
        s_dur = tomillisecond(self.local_end)-tomillisecond(self.local_start)
        
        
        best = self.video.getbestaudio() # you can get audio only!
        playurl = best.url
        Instance = vlc.Instance()
        player = Instance.media_player_new()
        Media = Instance.media_new(playurl)
        Media.get_mrl()
        player.set_media(Media)
        player.play()
        time.sleep(0.1)
        for x in range(0,count):
            vlc.libvlc_media_player_set_position(player,c_float(s_pos)) # set start point
            while True:
                if vlc.libvlc_media_player_get_time(player)>tomillisecond(self.local_end):
                    break;
        
        player.stop()
        
    #def eq(self):
        
    
class mixer():
    def __init__(self):
        self.songlist = songlist
 


#s = songBlock({"url":"https://www.youtube.com/watch?v=xoSRIi-SnDc","start":"00:00:13","end":"00:00:15"})
s2 = songBlock({"url":"https://www.youtube.com/watch?v=lKrJp9_UB4M","start":"00:04:00"})
#s.play()
s2.play()
#print(tomillisecond(s.local_end)-tomillisecond(s.local_start))
    

songblock lKrJp9_UB4M finished


In [ ]:
from multiprocessing import Pool
pool = Pool(processes=2)
pool.map(nonSelen.updateByTaxon,qList)

In [ ]:
n_eq=vlc.libvlc_audio_equalizer_get_preset_count()#vlc.libvlc_audio_equalizer_get_band_count() 
for idx in range(0,n_eq):
    print(vlc.libvlc_audio_equalizer_get_preset_name(idx))

In [12]:
import json
with open('./mixml.json') as f:
    mixml = json.load(f)
    
def compiler(mixml):
    # get all? songs --> later, some scheduler might needed
    # multi-track...? maybe...
    # if there is duplicate in songlist --> need to optimize this part
    songList = []
    for song in mixml['track1']:
        songList.append(songBlock(song))
    
    # play part:
    for song in songList:
        song.play()
        
    return 0


compiler(mixml)

{'url': 'https://www.youtube.com/watch?v=xoSRIi-SnDc', 'start': '00:00:00', 'end': '00:00:05', 'equalizer': 'Flat'}
songblock xoSRIi-SnDc finished
songblock xoSRIi-SnDc finished
songblock xoSRIi-SnDc finished
songblock xoSRIi-SnDc finished
songblock xoSRIi-SnDc finished


0